In [7]:
%run SharedParams.ipynb
%run MerkleTree.ipynb
import ast

proof = proof_example
commitments = []
fiat_shamir = FiatShamir()

In [8]:
# parse utils

def parse_arg(arg_name):
    data = proof.pop(0)
    assert data.startswith(arg_name + ":")
    data = data[len(arg_name) + 1:]
    return data

def parse_felt_and_commit(arg_name):
    data = F_gen * int(parse_arg(arg_name))
    commitments.append(int(data))
    return data

def parse_felt_and_auth(arg_name, index, merkle_root, tree_height):
    data = F_gen * int(parse_arg(arg_name))
    path = parse_arg("path")
    path = ast.literal_eval(path)
    MerkleTree.auth_path(data, index, path, merkle_root, tree_height)
    return data

def parse_and_commit_hash(arg_name):
    arg = bytes.fromhex(parse_arg(arg_name))
    commitments.append(arg)
    return arg

In [9]:
# parse trace polynomial eval root, generate cp betas, parse cp merkle root

eval_merkle_root = parse_and_commit_hash("eval_merkle")
beta0, beta1, beta2, beta3, beta4, beta5 = (fiat_shamir.get_random_int(commitments) % P for i in range(6))
cp_eval_merkle_root = parse_and_commit_hash("cp_eval_merkle")
oods_z = fiat_shamir.get_random_int(commitments) % P
cp_z = parse_felt_and_commit("cp_z")
mask_z1 = parse_felt_and_commit("mask_z1")
mask_z2 = parse_felt_and_commit("mask_z2")
mask_z3 = parse_felt_and_commit("mask_z3")
alpha1, alpha2, alpha3, alpha_cp = (fiat_shamir.get_random_int(commitments) % P for i in range(4))
oods_eval_merkle_root = parse_and_commit_hash("oods_eval_merkle")

In [10]:
# Compute cp_z from mask_z's, and assert it is correct

fib_constraint_poly_comp = (mask_z1 + mask_z2 - mask_z3) / (oods_z ** 1024 - 1)
for i in range(999, 1024):
    fib_constraint_poly_comp *= (oods_z - gamma ** i)

boundary_constraint_1_poly_comp = (mask_z1 - fib_1)/(oods_z - 1)
boundary_constraint_last_poly_comp = (mask_z1 - fib_last)/(oods_z - gamma ** 1000)


cp_comp = beta0 + beta1 * (fib_constraint_poly_comp * oods_z ** (1023 - fib_constraint_max_deg))
cp_comp += beta2 + beta3 * (boundary_constraint_1_poly_comp * oods_z ** (1023 - boundary_constraint_max_deg))
cp_comp += beta4 + beta5 * (boundary_constraint_last_poly_comp * oods_z ** (1023 - boundary_constraint_max_deg))

assert cp_comp == cp_z


In [11]:
# parse commitments on FRI layers and generate layers beta

layers_roots = []
layers_betas = []
for layer in range(int(math.log(gamma_order, 2)) - 1):
    layers_betas.append(fiat_shamir.get_random_int(commitments))
    layers_roots.append(parse_and_commit_hash("layer_root"))

# last layer
layers_betas.append(fiat_shamir.get_random_int(commitments))
last_layer_const = parse_felt_and_commit("last_layer_const")

In [12]:
def compute_oods_from_evals(trace_eval, cp_eval, index):
    oods_comp = alpha1 * (trace_eval-mask_z1)/((w * g ** index)-oods_z)
    oods_comp += alpha2 * (trace_eval-mask_z2)/((w * g ** index)- (gamma * oods_z))
    oods_comp += alpha3 * (trace_eval-mask_z3)/((w * g ** index)-(gamma ** 2 * oods_z))
    oods_comp += alpha_cp * (cp_eval-cp_z)/((w * g ** index)-oods_z)
    
    return F_gen * oods_comp
    

In [13]:
# get trace evaluations and cp_eval, test their compatibility and return cp_eval

def parse_evals_and_verify_oods(first_index: int, second_index: int):
    trace_eval = parse_felt_and_auth("trace_eval", first_index, eval_merkle_root, tree_height)
    cp_eval = parse_felt_and_auth("cp_eval", first_index, cp_eval_merkle_root, tree_height)
            
    oods_first_eval = parse_felt_and_auth("layer_first_eval", first_index, oods_eval_merkle_root, tree_height)
    oods_second_eval = parse_felt_and_auth("layer_second_eval", second_index, oods_eval_merkle_root, tree_height)
    
    # TODO - check the trace and cp against the oods values. (build polynom)
    
    assert oods_first_eval == compute_oods_from_evals(trace_eval, cp_eval, first_index)
    
    
    
    return oods_first_eval, oods_second_eval

In [14]:
def verify_fri_layer(layer_size, dilution, cur_index, layer_first_index, layer_second_index, layer_first_eval, layer_second_eval, next_eval_index):
    plus_eval = layer_first_eval if layer_first_index < layer_second_index else layer_second_eval
    minus_eval = layer_first_eval if layer_first_index > layer_second_index else layer_second_eval
        
    even_part = (F_gen * plus_eval + minus_eval) / 2
    odd_part = (F_gen * plus_eval - minus_eval) / (2 * w**dilution * g**(cur_index))
        
    if next_eval_index == len(layers_roots):
        expected_value = F_gen * last_layer_const
    else:
        expected_value = F_gen * layers_first_eval[next_eval_index]
    assert expected_value == even_part + layers_betas[next_eval_index] * odd_part
    

In [15]:
# parse queries and test their FRI validity

for i in range(N_FRI_QUERIES):
    print(f"running Fri query {i}")
    tree_height = int(math.log(g_order, 2))    
    first_index = fiat_shamir.get_random_int(commitments) % g_order
    second_index = (first_index + int(g_order / 2)) % g_order
    oods_first_eval, oods_second_eval = parse_evals_and_verify_oods(first_index, second_index)
        
    # Get and authenticate the plus and minus evaluations.
    layers_first_eval = []
    layers_second_eval = []
    layer_size = g_order
    for layer_root in layers_roots:
        tree_height -= 1
        layer_size = int(layer_size / 2)
        layer_first_index = first_index % layer_size
        layer_second_index = int(first_index - int(layer_size / 2)) % layer_size
        layers_first_eval.append(parse_felt_and_auth("layer_first_eval", layer_first_index, layer_root, tree_height))
        layers_second_eval.append(parse_felt_and_auth("layer_second_eval", layer_second_index, layer_root, tree_height))

    # verify first fri layer (oods evaluation)
    verify_fri_layer(
        layer_size=g_order,
        dilution=1,
        cur_index=first_index % int(g_order / 2),
        layer_first_index=first_index,
        layer_second_index=second_index,
        layer_first_eval=oods_first_eval,
        layer_second_eval=oods_second_eval,
        next_eval_index=0
    )

    
    # Check Fri layers evaluations.
    dilution = 2
    for layer_index in range(len(layers_roots)):
        layer_size=int(g_order / dilution)
        verify_fri_layer(
            layer_size=layer_size,
            dilution=dilution,
            cur_index=(dilution * first_index) % int(g_order / 2),
            layer_first_index=first_index % layer_size,
            layer_second_index=int(first_index - int(layer_size / 2)) % layer_size,
            layer_first_eval=layers_first_eval[layer_index],
            layer_second_eval=layers_second_eval[layer_index],
            next_eval_index=layer_index+1
        )
            
        dilution *= 2

        
assert len(proof) == 0
print("Amen :)")

running Fri query 0
running Fri query 1
Amen :)
